In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Constants
dt        = 1.                           # year        Time step
Al_ox     = 1000.                        # mg/kg
Fe_ox     = 1400.                        # mg/kg
Al_Fe_ox  = Al_ox/26.982 + Fe_ox/55.847  # mmol/kg
SOM       = 4.                           # %           Soil Organic Matter content
DBD       = 1000./(0.6+3.42*SOM/100.)    # kg soil/m3  Dry bulk density
h_soil    = 0.25                         # m
m_soil    = 100 * 100 * h_soil * DBD     # kg soil/ha
p_surplus = 3.0e6                        # l/ha/yr     precipitation surplus
Y_veg_max = 10000.                       # kg DM/ha/yr Maximum yield of crop/vegetation/grass
C_P_veg   = 4.1                          # g P/kg Dry Matter of vegetation Phosphorus content of the vegetation
Q_P_fert  = 35.0e6                       # mg P/ha/yr  Phosphorus Fertilisation flux
Q_P_atm   =  1.0e6                       # mg P/ha/yr  Phosphorus Atmospheric Deposition

# Parameters
nyears    = 750
t         = range(nyears)


# Initials conditions
C_P_ox_ini= 700.  # mg P/kg soil


# Array initialisation
PSD       = np.zeros((nyears)) * np.nan   # %              PSD
TDP       = np.zeros((nyears)) * np.nan   # mg TDP/l       Total Dissolved Phosphorus
C_P_ox    = np.zeros((nyears))            # mg P/kg soil   P concentration in soil 
P_ox      = np.zeros((nyears))            # mg P/ha        Total P in soil
Qk_P_out  = np.zeros((nyears))            # kg P/ha/yr P   Leaching flux
RY        = np.zeros((nyears))            # %              Relative Yield
Qh_P      = np.zeros((nyears))            # kg P/ha/yr P   Harvest flux
P_Al      = np.zeros((nyears))            # mg P2O5/100 g

# Initialisation (first time step)
C_P_ox[0]          = C_P_ox_ini           # mg P/kg soil
P_ox  [0]          = C_P_ox[0] * m_soil   # mg P in the soil layer

# Time integration
for it in range(1,nyears):
    
    # diagnostics
    PSD     [it] = 100* (C_P_ox[it-1] / 30.974) / (0.5*Al_Fe_ox)      # % Phosphorus Saturation Degree
    logPSD       = np.log10(PSD[it])

    TDP     [it] = np.power(10,-2.26 + 1.23 * logPSD )                # mg TDP/l
    RY      [it] = 23+(100-23)*(1-np.exp(-0.09*PSD[it]))              # %

    # fluxes
    Qk_P_out[it] = p_surplus * TDP[it]                                # mg TDP/ha/yr (l/ha/yr * mg TDP/l)
    Qh_P    [it] = 1000*0.01*RY[it] * Y_veg_max * C_P_veg             # mg P/ha/yr
    P_Al    [it] = 1.32 * PSD[it] - 1.63
    
    # P-balance
    dPdt         = Q_P_fert + Q_P_atm - Qk_P_out[it] - Qh_P[it]       # mg P/ha/yr
    P_ox  [it]   = P_ox[it-1] + dPdt * dt                             # mg P/ha
    C_P_ox[it]   = P_ox[it]/m_soil                                    # mg P/kg soil

#   if np.mod(it,20)==1:
#       print(' it  Pox  C_P_ox PSD  logPSD   TDP leach yield    RY  dPdt')
#   print('%3d %5.3f %5.5f %5.5f %5.5f %5.5f %5.5f %5.5f %5.5f %5.5f'%(it,P_ox[it]*1e-6,C_P_ox[it],PSD[it],logPSD,TDP[it],Qk_P_out[it]*1e-6,Qh_P[it]*1e-6,RY[it],dPdt*1e-6))
    
# Output as figure    
f = plt.figure(1)
ax = f.add_subplot(111)
ax.plot(t,PSD)